# Neural Networks

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import imblearn
print(imblearn.__version__)

ModuleNotFoundError: No module named 'numpy'

In [2]:
data = pd.read_csv("churn.csv")
data = data.fillna(0)
del data['TENURE']
data.head()

,MSISDN,FLAG,LAST_EXPIRY_RECHARGE_DIST,LAST_DISTANCE_ACHAT,LAST_DISTANCE_ENTRANT,LAST_DISTANCE_SORTANT,LAST_DISTANCE_RECHARGE,LAST_MONTANT_RECHARGE,LAST_DISTANCE_SOS,LAST_DECONNEXION,...,NB_APPEL_ENTRANT,NB_APPEL_SORTANT,NB_RECHARGE,NB_SOS,LIFE_TIME,MONTANT_DATA,MONTANT_RECHARGE,MONTANT_SOS,NB_MINUTES,RATIO_APPEL_ENTRANT
0,-382811781,0,20,0.0,6.0,5.0,92.0,1.0,23.0,-1.0,...,173.0,39.0,7.0,4.0,862.0,0.0,5.0,11.0,107,0.65
1,-1298385162,0,73,0.0,20.0,28.0,31.0,4.4,31.0,9.0,...,804.0,42.0,5.0,4.0,846.0,0.0,16.0,14.0,387,0.69
2,-474135192,0,96,11.0,9.0,5.0,14.0,10.0,11.0,2.0,...,732.0,429.0,61.0,25.0,392.0,57.0,194.0,101.0,115,0.44
3,-472538783,0,-36,0.0,7.0,82.0,83.0,1.0,83.0,3.0,...,174.0,94.0,7.0,6.0,152.0,0.0,2.0,9.0,134,0.32
4,-227903079,0,-131,0.0,5.0,12.0,17.0,1.0,0.0,-1.0,...,100.0,2.0,1.0,0.0,455.0,0.0,1.0,0.0,69,0.81


In [4]:
df = data[['FLAG', 'LAST_DECONNEXION', 'MAX_BTW_DECONNEXION', 'MIN_BTW_DECONNEXION' , 'MIN_BTW_RECHARGE' , 'MONTANT_RECHARGE' , 'MONTANT_DATA' , 'LAST_DISTANCE_ENTRANT' , 'LAST_DISTANCE_ACHAT' , 'LAST_DISTANCE_SOS' , 'LAST_DISTANCE_RECHARGE' , 'LAST_DISTANCE_SORTANT' , 'RATIO_APPEL_ENTRANT' ]]
df = df.fillna(0)
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values
X = np.where(X<0, 0, X)
df["FLAG"].value_counts()

1    117586
0      5001
Name: FLAG, dtype: int64

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)
print(X)

[[-0.16231508  0.62389331 -0.25072438 ...  2.36165291 -0.65576968
   0.93722353]
 [ 1.38487459 -0.14739633 -0.25072438 ...  0.07376977  0.5435983
   1.19477015]
 [ 0.18150485 -0.2759446   0.17778002 ... -0.56383701 -0.65576968
  -0.41489625]
 ...
 [-0.16231508 -0.2759446  -0.25072438 ... -0.90139354 -0.70791612
   0.29335697]
 [-0.16231508 -0.2759446   0.17778002 ... -0.75136841 -0.49933038
   0.29335697]
 [-0.16231508  3.70905187  2.10604985 ... -0.45131817 -0.08215891
  -0.80121619]]


In [7]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(sampling_strategy=1) 
X, y = ros.fit_resample(X, y)
df["FLAG"].value_counts()

1    117586
0      5001
Name: FLAG, dtype: int64

In [8]:
import numpy as np
unique, counts = np.unique(y, return_counts=True)
print(np.asarray((unique, counts)).T)

[[     0 117586]
 [     1 117586]]


In [9]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [10]:
from xgboost import XGBClassifier
booster = XGBClassifier(max_depth = 88, min_samples_leaf = 10 , 
                        min_samples_split = 2, n_estimators = 68)
booster.fit(X_train, y_train)

[22:31:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "min_samples_leaf", "min_samples_split" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=88, max_leaves=0, min_child_weight=1,
              min_samples_leaf=10, min_samples_split=2, missing=nan,
              monotone_constraints='()', n_estimators=68, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [44]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
y_pred_booster = booster.predict(X_test)
cm = confusion_matrix(y_test, y_pred_booster)
print(cm)
accuracy_score(y_test,y_pred_booster)

[[23523     9]
 [  523 22980]]


0.9886892739449347

In [45]:
f1_score(y_test,y_pred_booster)

0.9885571711262152

In [12]:
# Predicting the Test set results
y_pred = booster.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [1 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [1 1]]


In [41]:
print(booster.predict(sc.transform([[-1.0,5.0,3.0,-1.0,1.0,0.0,5.0,0.0,0.0,17.0,12.0,0.81]])) ) 
print(booster.predict(sc.transform([[2.0,5.0,5.0,1.0,194.0,57.0,9.0,11.0,11.0,14.0,5.0,0.44]])) )

# ['LAST_DECONNEXION', 'MAX_BTW_DECONNEXION', 
# 'MIN_BTW_DECONNEXION' , 'MIN_BTW_RECHARGE' , 'MONTANT_RECHARGE' , 
# 'MONTANT_DATA' , 'LAST_DISTANCE_ENTRANT' , 
# 'LAST_DISTANCE_ACHAT' , 'LAST_DISTANCE_SOS' , 'LAST_DISTANCE_RECHARGE' , 
# 'LAST_DISTANCE_SORTANT' , 'RATIO_APPEL_ENTRANT' ]

#12 



[1]
[1]


In [15]:
df = df.rename({"FLAG": "CHURNED"}, axis=1)
df = df.rename({"LAST_DECONNEXION": "Last Disconnect Time"}, axis=1)
df = df.rename({"MAX_BTW_DECONNEXION": "Maximum Time Between Disconnections"}, axis=1)
df = df.rename({"MIN_BTW_DECONNEXION": "Minimum Time Between Disconnections"}, axis=1)
df = df.rename({"MIN_BTW_RECHARGE": "Maximum Time Between Recharges"}, axis=1)
df = df.rename({"MONTANT_RECHARGE": "Phone Top-up Amount"}, axis=1)
df = df.rename({"MONTANT_DATA": "4G Data Amount"}, axis=1)
df = df.rename({"LAST_DISTANCE_ENTRANT": "The Last Day the Customer Made an Incoming Call"}, axis=1)
df = df.rename({"LAST_DISTANCE_ACHAT": "The Last Day the Customer Made a Purchase"}, axis=1)
df = df.rename({"LAST_DISTANCE_SOS": "The Last Day the Customer Made a Purchase of SOS Credit"}, axis=1)
df = df.rename({"LAST_DISTANCE_RECHARGE": "The Last Day the Customer Made a Recharge "}, axis=1)
df = df.rename({"LAST_DISTANCE_SORTANT": "The Last Day the Customer Made an Outgoing Call"}, axis=1)
df = df.rename({"RATIO_APPEL_ENTRANT": "Incoming Put-Call Ratio "}, axis=1)

df.head()

,CHURNED,Last Disconnect Time,Maximum Time Between Disconnections,Minimum Time Between Disconnections,Maximum Time Between Recharges,Phone Top-up Amount,4G Data Amount,The Last Day the Customer Made an Incoming Call,The Last Day the Customer Made a Purchase,The Last Day the Customer Made a Purchase of SOS Credit,The Last Day the Customer Made a Recharge,The Last Day the Customer Made an Outgoing Call,Incoming Put-Call Ratio
0,0,-1.0,12.0,3.0,4.0,5.0,0.0,6.0,0.0,23.0,92.0,5.0,0.65
1,0,9.0,6.0,3.0,1.0,16.0,0.0,20.0,0.0,31.0,31.0,28.0,0.69
2,0,2.0,5.0,5.0,1.0,194.0,57.0,9.0,11.0,11.0,14.0,5.0,0.44
3,0,3.0,5.0,5.0,1.0,2.0,0.0,7.0,0.0,83.0,83.0,82.0,0.32
4,0,-1.0,5.0,3.0,-1.0,1.0,0.0,5.0,0.0,0.0,17.0,12.0,0.81


In [16]:
df.describe()

,CHURNED,Last Disconnect Time,Maximum Time Between Disconnections,Minimum Time Between Disconnections,Maximum Time Between Recharges,Phone Top-up Amount,4G Data Amount,The Last Day the Customer Made an Incoming Call,The Last Day the Customer Made a Purchase,The Last Day the Customer Made a Purchase of SOS Credit,The Last Day the Customer Made a Recharge,The Last Day the Customer Made an Outgoing Call,Incoming Put-Call Ratio
count,122587.000000,122587.000000,122587.000000,122587.000000,122587.000000,122587.000000,122587.000000,122587.000000,122587.000000,122587.000000,122587.000000,122587.000000,122587.000000
mean,0.959204,0.015948,7.146622,4.170230,3.639187,42.596409,24.449077,8.332735,18.360234,17.060773,29.033136,17.575542,0.504438
std,0.197817,5.971368,7.779211,4.667416,9.243888,50.239161,42.542579,8.157228,24.142698,22.795884,26.662310,19.176845,0.155312
min,0.000000,-1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,-1.000000,5.000000,3.000000,1.000000,9.000000,0.000000,3.000000,0.000000,0.000000,5.000000,3.000000,0.400000
50%,1.000000,-1.000000,5.000000,3.000000,1.000000,26.000000,6.000000,6.000000,7.000000,6.000000,23.000000,11.000000,0.490000
75%,1.000000,-1.000000,5.000000,5.000000,2.000000,58.000000,30.000000,12.000000,29.000000,27.000000,45.000000,26.000000,0.600000
max,1.000000,99.000000,98.000000,98.000000,100.000000,966.000000,1320.000000,100.000000,100.000000,100.000000,100.000000,100.000000,0.990000


In [36]:
X = np.array([[2.0,5.0,5.0,1.0,194.0,57.0,9.0,11.0,11.0,14.0,5.0,0.44]]) #NOT CHURNED : 0
X = X.astype(float)
X

array([[-1.  ,  5.  ,  3.  , -1.  ,  1.  ,  0.  ,  5.  ,  0.  ,  0.  ,
        17.  , 12.  ,  0.81]])

In [37]:
y_pred = booster.predict(sc.transform((X)))
y_pred 

array([1])

In [47]:
data = {"model": booster, "scaler" : sc}
with open('saved_steps.pkl', 'wb') as file:
    pickle.dump(data, file)

In [48]:
with open('saved_steps.pkl', 'rb') as file:
    data = pickle.load(file)

booster = data["model"]

In [49]:
y_pred = booster.predict(sc.transform((X)))
y_pred > 0.5
y_pred

array([1])